In [ ]:
import math

def compute_metaldecking(data, fc=20.7, fy=276, cover=20):
    '''
    fc = Compressive Concrete Strength 20.68 MPa
    fy = Steel Bar Yield Strength 276 MPa
    '''
    MM_TO_M = 0.001
    UW_CONC = 23.56
    UW_STEEL = 77
    WIDTH = 1000  # width of panel / tributary width to compute uniform load
    BEAM_WIDTH = 350  # width of beam / to compute clear span
    EFFECTIVE_WIDTH = 0.995 # https://atepsmr.tripod.com/dns-deck4.html

    # --------------------------------------------------------------------------
    # Total Nominal Width of Steel Deck DN-4 in mm
    # --------------------------------------------------------------------------
    if data[5] == 0.8:
        nominal_width = 1021
    elif data[5]  == 1.0:
        nominal_width = 1276
    elif data[5] == 1.2:
        nominal_width = 1532
    elif data[5] == 1.6:
        nominal_width = 2042
    else:
        raise ValueError("Invalid steel deck thickness")

    # """ uncomment for checking with excel """
    # nominal_width = 1219

    # --------------------------------------------------------------------------
    # "Not Applicable" or "Input Beam Width"
    # --------------------------------------------------------------------------
    if data[2] == 1:
        widthbeam = 0  # not applicable
    else:
        widthbeam = BEAM_WIDTH

    # --------------------------------------------------------------------------
    # Compute slab permanent clear span in mm
    # --------------------------------------------------------------------------
    if data[9] == 0:
        if data[2] == 1:
            span = data[1]
        else:
            span = data[1] - widthbeam
    else:
        if data[2] == 1:
            span = data[1] / data[9]
        else:
            span = (data[1] / data[9]) - widthbeam

    # --------------------------------------------------------------------------
    # Compute uniform live load per meter strip of slab in kN/m
    # wll = (LL in kN/m^2) × (f) × (panel width in mm) × (0.001)
    # --------------------------------------------------------------------------
    wll = data[7] * data[11] * WIDTH * MM_TO_M

    # --------------------------------------------------------------------------
    # Compute distance of top bar from bottom of slab
    # (slab thickness - cover - topbar/2)
    # --------------------------------------------------------------------------
    dprime = data[4] - cover - data[15] / 2

    # --------------------------------------------------------------------------
    # Section calculated data
    # --------------------------------------------------------------------------
    asd = nominal_width * data[5] / EFFECTIVE_WIDTH 
    abb = (data[13] / EFFECTIVE_WIDTH) * math.pi * math.pow(data[12], 2) / 4
    aconc = data[4] * 1000 - 18480 #TODO 18480
    es = 200000
    ec = 4700 * math.sqrt(fc)
    nmod = es / ec

    # moment of inertia of steel deck per meter
    # if data[5] == 0.8:
    #     isd = 304581.947
    # elif data[5] == 1.0:
    #     isd = 373367.871
    # elif data[5] == 1.2:
    #     isd = 442153.796
    # elif data[5] == 1.6:
    #     isd = 579725.646
    
    #TODO TO CONFIRM WITH SIR ALBERT
    if data[5] == 0.8:
        isd = 518000
    elif data[5] == 1.0:
        isd = 648000
    elif data[5] == 1.2:
        isd = 777000
    elif data[5] == 1.6:
        isd = 1037000
    else:
        raise ValueError("Invalid steel deck thickness")

    sn = isd / (50 - 18.333)
    sp = isd / 18.333
    wsd = UW_STEEL * asd / 1000000
    wconc = aconc * UW_CONC / 1000000
    wsidl = data[6] + (data[3] / 1000) * 23.56
    wcl = 1.4715
    w1 = 1.6 * wconc + 1.2 * wsd + 1.4 * wcl
    wp = 1.6 * wll + 1.2 * (wsidl + wconc + wsd)

    # --------------------------------------------------------------------------
    # Temporary props
    # --------------------------------------------------------------------------
    spanprop = span / (data[10] + 1)
    fip1 = 0.125 * w1 * math.pow(spanprop, 2) / 0.95 / sn
    fip2 = 0.08 * w1 * math.pow(spanprop, 2) / 0.95 / sn

    if data[10] == 0:
        fipa = fip1
    else:
        fipa = fip2

    if fipa < 0.95 * data[8]:
        fip = fipa
    else:
        fip = data[8]

    # --------------------------------------------------------------------------
    # One meter strip cracked slab section for positive bending
    # --------------------------------------------------------------------------
    ax = WIDTH / (2 * nmod)
    bx = asd + abb
    cx = asd * (data[4] - 22.149) + abb * (data[4] - data[14])
    ac = (-bx + math.sqrt(bx**2 + 4 * ax * cx)) / (2 * ax)

    ic = (
        (1000 / 3 / nmod) * ac**3
        + isd
        + asd * (data[4] - ac - 22.149) ** 2
        + abb * (data[4] - ac - data[14]) ** 2
    )
    sc = ic / (data[4] - ac)

    # --------------------------------------------------------------------------
    # One meter strip uncracked slab section for positive bending
    # --------------------------------------------------------------------------
    b2 = 534
    aux = (WIDTH - b2) / (2 * nmod)
    bux = (asd + abb) + (b2 * data[4] / nmod)
    cux = (
        asd * (data[4] - 22.149)
        + abb * (data[4] - data[14])
        + b2 * math.pow(data[4], 2) / (2 * nmod)
    )
    auc = (-bux + math.sqrt(bux**2 + 4 * aux * cux)) / (2 * aux)

    iuc = (
        (1000 / 3 / nmod) * math.pow(auc,3)
        + asd * math.pow(data[4] - auc - 22.149,2)
        + abb * math.pow(data[4] - auc - data[14],2)
        + (534 / 3 / nmod) * math.pow(data[4] - auc,3)
    )
    suc = iuc / 22.149

    # --------------------------------------------------------------------------
    # Average Cracked and Uncracked Moment of Inertia
    # --------------------------------------------------------------------------
    iave = (iuc + ic) / 2

    # --------------------------------------------------------------------------
    # Moments
    # --------------------------------------------------------------------------
    mp_support = data[16] * wp * math.pow((span / 1000),2)
    if data[9] == 0:
        mp_midspan = 0.125 * wp * math.pow((span / 1000),2)
    else:
        mp_midspan = 0.08 * wp * math.pow((span / 1000),2)

    # --------------------------------------------------------------------------
    # Concrete Stress
    # --------------------------------------------------------------------------
    fc_calc = mp_midspan * ac / ic / nmod * 1_000_000
    allowablefc = 0.85 * fc
    concretestress = fc_calc / allowablefc
    if concretestress < 1:
        note_conc = "SAFE"
    else:
        note_conc = "FAIL"

    # --------------------------------------------------------------------------
    # Steel Deck Stress
    # --------------------------------------------------------------------------
    fsd = mp_midspan * 1_000_000 * (data[4] - ac) / ic
    allowablefsd = (0.85 * data[8] - fip)
    if allowablefsd < 0:
        steeldeckstress = fsd / allowablefsd * (-1)
    else:
        steeldeckstress = fsd / allowablefsd
    if steeldeckstress < 1:
        note_steeldeck = "SAFE"
    else:
        note_steeldeck = "FAIL"

    # --------------------------------------------------------------------------
    # Steel Bar Stress
    # --------------------------------------------------------------------------
    fs = mp_midspan * 1_000_000 * (data[4] - ac - data[14]) / ic
    allowablefs = 0.9 * fy
    steelbarstress = fs / allowablefs
    if steelbarstress < 1:
        note_steelbar = "SAFE"
    else:
        note_steelbar = "FAIL"

    # --------------------------------------------------------------------------
    # Check Deflection in mm
    # --------------------------------------------------------------------------
    deflection_LL = 0.013 * wll * math.pow(span, 4) / (es * iave)
    allowable_deflection_LL = span / 360
    ratio_LL = deflection_LL / allowable_deflection_LL
    if ratio_LL < 1:
        note_deflection_LL = "SAFE"
    else:
        note_deflection_LL = "FAIL"

    deflection = 0.013 * (wll + wsd + wconc + wsidl) * math.pow(span, 4) / (es * iave)
    allowable_deflection = span / 240
    ratio = deflection / allowable_deflection
    if ratio < 1:
        note_deflection = "SAFE"
    else:
        note_deflection = "FAIL"

    # --------------------------------------------------------------------------
    # Design of Negative Reinforcing
    # --------------------------------------------------------------------------
    ax_neg = fy / (2 * 0.85 * fc * 500)
    bx_neg = -dprime
    cx_neg = mp_support * 1_000_000 / 0.9 / fy
    as_neg = (-bx_neg - math.sqrt(math.pow(bx_neg,2) - 4 * ax_neg * cx_neg)) / (2 * ax_neg)
    if data[9] == 0:
        nbar_neg = 0
    else:
        nbar_neg = as_neg / (math.pi * (data[15] ** 2) / 4)

    if nbar_neg == 0:
        sneg = 0
    else:
        sneg = 1000 / nbar_neg

    aww = 0.002 * aconc
    nww = aww / (math.pi * math.pow(data[17], 2) / 4)
    sww = 1000 / nww

    results = {
        "widthbeam": widthbeam,
        "span": span,
        "wll": wll,
        "dprime": dprime,
        "isd": isd,
        "sn": sn,
        "sp": sp,
        "wsd": wsd,
        "wconc": wconc,
        "wsidl": wsidl,
        "wcl": wcl,
        "w1": w1,
        "wp": wp,
        "spanprop": spanprop,
        "fip1": fip1,
        "fip2": fip2,
        "fip": fip,
        "ax": ax,
        "bx": bx,
        "cx": cx,
        "ac": ac,
        "ic": ic,
        "sc": sc,
        "aux": aux,
        "bux": bux,
        "cux": cux,
        "auc": auc,
        "iuc": iuc,
        "suc": suc,
        "iave": iave,
        "mp_support": mp_support,
        "mp_midspan": mp_midspan,
        "fc": fc_calc,
        "allowablefc": allowablefc,
        "concretestress": concretestress,
        "note_concretestress": note_conc,
        "fsd": fsd,
        "allowablefsd": allowablefsd,
        "steeldeckstress": steeldeckstress,
        "note_steeldeck": note_steeldeck,
        "fs": fs,
        "allowablefs": allowablefs,
        "steelbarstress": steelbarstress,
        "note_steelbar": note_steelbar,
        "deflection_LL": deflection_LL,
        "allowable_deflection_LL": allowable_deflection_LL,
        "ratio_LL": ratio_LL,
        "note_deflection_LL": note_deflection_LL,
        "deflection": deflection,
        "allowable_deflection": allowable_deflection,
        "ratio": ratio,
        "note_deflection": note_deflection,
        "ax_neg": ax_neg,
        "bx_neg": bx_neg,
        "cx_neg": cx_neg,
        "as_neg": as_neg,
        "nbar_neg": nbar_neg,
        "sneg": sneg,
        "aww": aww,
        "nww": nww,
        "sww": sww,
    }
    return results


if __name__ == "__main__":
    
    fysd = 550
    beam_type = 1
    steel_deck_thk = 0.8
    slab_thk = 125
    
    slab_data = [
        #   name[0]
        #   length[1] - total length of panel in mm
        #   beam_type[2] - type of beam: 1 = concrete beam ; 2 = steel beam
        #   topping[3] - concrete topping thickness in mm
        #   slab_thk[4] - slab thickness in mm
        #   steel_deck_thk[5] - steel deck thickness in mm
        #   sdl[6] - superimposed dead load in kN/m^2
        #   liveload[7] - live load in kN/m^2
        #   fysd[8] - steel deck yield strength in MPa
        #   nspan[9] - number of continuous (permanent) spans
        #   nprop[10] - number of intermediate temporary props (per span during pouring)
        #   f_vibration[11] - vibration factor
        #   botbar[12] - bottom bar diameter in mm
        #   nbot[13] - number of bottom bars per one steel deck panel
        #   d_botbar[14] - distance of bottom steel bar in mm
        #   topbar[15] - negative bars > top bar diameter at near support in mm
        #   coef[16] - moment coefficient at support
        #   dww[17] - temperature bars > diameter of distribution steel in mm
        # ]
        ['A', 3287, beam_type, 20, slab_thk, steel_deck_thk, 1.52, 1.9,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['B', 1725, beam_type, 50, slab_thk, steel_deck_thk, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['C', 2356, beam_type, 50, slab_thk, steel_deck_thk, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['D', 2356, beam_type, 50, slab_thk, steel_deck_thk, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['E', 1500, beam_type, 50, slab_thk, steel_deck_thk, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['F', 1725, beam_type, 50, slab_thk, steel_deck_thk, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['G', 1725, beam_type, 50, slab_thk, steel_deck_thk, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['H', 2356, beam_type, 50, slab_thk, steel_deck_thk, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['I', 2356, beam_type, 50, slab_thk, steel_deck_thk, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
        ['J', 1500, beam_type, 50, slab_thk, steel_deck_thk, 1.52, 4.8,  fysd, 1, 2, 1, 0, 0, 25, 12, 0.1, 10],
     ]

    # print (compute_metaldecking(slab_data[0])) - uncomment for checking with excel

    i = 0
    sd_length = len(slab_data)
    while i < sd_length:
        name = slab_data[i][0]
        results = compute_metaldecking(slab_data[i])
        
        note_steeldeck      = results["note_steeldeck"]
        note_concretestress = results["note_concretestress"]
        note_steelbar       = results["note_steelbar"]
        note_deflection_LL  = results["note_deflection_LL"]
        note_deflection     = results["note_deflection"]
        nbar_neg = math.ceil(results["nbar_neg"])
        sneg = math.ceil(results["sneg"])
        nww = math.ceil(results["nww"])
        sww = math.ceil(results["sww"])
        
        # Print all values in one formatted line
        print(name + " ; SD = " + str(note_steeldeck) +
            "; CONCRETE = " + str(note_concretestress) +
            "; STEEL_BAR = " + str(note_steelbar) +
            "; DEF_LL = " + str(note_deflection_LL) +
            "; DEF_COMBINED = " + str(note_deflection))

        print("    == NEGATIVE BARS ==\n" +
            "    Number of Bars = " + str(nbar_neg) +
            "; Spacing = " + str(sneg) +
            "; Required Spacing = 300")
        
        print("    == TEMPERATURE BARS ==\n" +
            "    Number of Bars = " + str(nww) +
            "; Spacing = " + str(sww) +
            "; Required Spacing = 300")
        
        i += 1


A ; SD = SAFE; CONCRETE = SAFE; STEEL_BAR = SAFE; DEF_LL = SAFE; DEF_COMBINED = SAFE
    == NEGATIVE BARS ==
    Number of Bars = 4; Spacing = 283; Required Spacing = 300
    == TEMPERATURE BARS ==
    Number of Bars = 3; Spacing = 369; Required Spacing = 300
B ; SD = SAFE; CONCRETE = SAFE; STEEL_BAR = SAFE; DEF_LL = SAFE; DEF_COMBINED = SAFE
    == NEGATIVE BARS ==
    Number of Bars = 2; Spacing = 649; Required Spacing = 300
    == TEMPERATURE BARS ==
    Number of Bars = 3; Spacing = 369; Required Spacing = 300
C ; SD = SAFE; CONCRETE = SAFE; STEEL_BAR = SAFE; DEF_LL = SAFE; DEF_COMBINED = SAFE
    == NEGATIVE BARS ==
    Number of Bars = 3; Spacing = 339; Required Spacing = 300
    == TEMPERATURE BARS ==
    Number of Bars = 3; Spacing = 369; Required Spacing = 300
D ; SD = SAFE; CONCRETE = SAFE; STEEL_BAR = SAFE; DEF_LL = SAFE; DEF_COMBINED = SAFE
    == NEGATIVE BARS ==
    Number of Bars = 3; Spacing = 339; Required Spacing = 300
    == TEMPERATURE BARS ==
    Number of Bars = 3